Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [7]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [8]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [9]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [18]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [47]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  reg_weight = 5e-3
  regularizers = tf.nn.l2_loss(weights)# + tf.nn.l2_loss(biases)
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 
   reg_weight * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [48]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# Minibatch loss at step 3000: 0.913319
# Minibatch accuracy: 75.8%
# Validation accuracy: 78.6%
# Test accuracy: 85.7%

# Minibatch loss at step 3000: 0.771258
# Minibatch accuracy: 81.2%
# Validation accuracy: 81.0%
# Test accuracy: 89.1%

Initialized
Minibatch loss at step 0: 32.956875
Minibatch accuracy: 8.6%
Validation accuracy: 13.4%
Minibatch loss at step 500: 1.838749
Minibatch accuracy: 78.1%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 0.835731
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 0.594025
Minibatch accuracy: 85.9%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 0.673939
Minibatch accuracy: 82.0%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.794670
Minibatch accuracy: 78.9%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.777917
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Test accuracy: 89.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [63]:
batch_size = 20

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  reg_weight = 5e-3
  regularizers = tf.nn.l2_loss(weights)# + tf.nn.l2_loss(biases) # No need to regularize biases
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 
   reg_weight * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [64]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# Minibatch loss at step 3000: 0.913319
# Minibatch accuracy: 75.8%
# Validation accuracy: 78.6%
# Test accuracy: 85.7%

# Minibatch loss at step 3000: 0.777917
# Minibatch accuracy: 81.2%
# Validation accuracy: 81.2%
# Test accuracy: 89.2%

Initialized
Minibatch loss at step 0: 29.686096
Minibatch accuracy: 10.0%
Validation accuracy: 8.1%
Minibatch loss at step 500: 2.864859
Minibatch accuracy: 55.0%
Validation accuracy: 71.5%
Minibatch loss at step 1000: 0.875005
Minibatch accuracy: 70.0%
Validation accuracy: 76.1%
Minibatch loss at step 1500: 0.699595
Minibatch accuracy: 75.0%
Validation accuracy: 70.8%
Minibatch loss at step 2000: 0.875005
Minibatch accuracy: 80.0%
Validation accuracy: 71.3%
Minibatch loss at step 2500: 0.994780
Minibatch accuracy: 70.0%
Validation accuracy: 70.7%
Minibatch loss at step 3000: 1.206748
Minibatch accuracy: 85.0%
Validation accuracy: 76.5%
Test accuracy: 85.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [61]:
batch_size = 20

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  reg_weight = 5e-3
  regularizers = tf.nn.l2_loss(weights)# + tf.nn.l2_loss(biases)
  
  # Training computation.
  logits = tf.matmul(tf.nn.dropout(tf_train_dataset, 0.5, seed=123), weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [62]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# vanilla
# Minibatch loss at step 3000: 0.913319
# Minibatch accuracy: 75.8%
# Validation accuracy: 78.6%
# Test accuracy: 85.7%

# l2 regularize
# Minibatch loss at step 3000: 0.777917
# Minibatch accuracy: 81.2%
# Validation accuracy: 81.2%
# Test accuracy: 89.2%

# dropout
# Minibatch loss at step 3000: 0.842658
# Minibatch accuracy: 78.9%
# Validation accuracy: 80.6%
# Test accuracy: 87.9%

Initialized
Minibatch loss at step 0: 27.415707
Minibatch accuracy: 5.0%
Validation accuracy: 16.7%
Minibatch loss at step 500: 8.363426
Minibatch accuracy: 60.0%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 0.511746
Minibatch accuracy: 90.0%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 0.403719
Minibatch accuracy: 95.0%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 2.144110
Minibatch accuracy: 80.0%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 2.266392
Minibatch accuracy: 65.0%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 2.358730
Minibatch accuracy: 75.0%
Validation accuracy: 78.4%
Test accuracy: 87.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [88]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Hidden Layer
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))
  logits_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1)
                        + biases_1)

  # Output Layer
  weights_2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  logits_2 = tf.matmul(tf.nn.dropout(logits_1, 0.8, seed=123), weights_2) + biases_2

  reg_weight = 5e-3
  regularizers = tf.nn.l2_loss(weights_2)# + tf.nn.l2_loss(biases)
  
  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_train_labels)
    + reg_weight * regularizers)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_2)
    
  hidden_validation = tf.nn.relu(
    tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_2) + biases_2)
    
  hidden_test = tf.nn.relu(
    tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(
    tf.matmul(hidden_test, weights_2) + biases_2)

In [89]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# vanilla lr
# Minibatch loss at step 3000: 0.913319
# Minibatch accuracy: 75.8%
# Validation accuracy: 78.6%
# Test accuracy: 85.7%

# l2 regularize lr
# Minibatch loss at step 3000: 0.777917
# Minibatch accuracy: 81.2%
# Validation accuracy: 81.2%
# Test accuracy: 89.2%

# dropout lr
# Minibatch loss at step 3000: 0.842658
# Minibatch accuracy: 78.9%
# Validation accuracy: 80.6%
# Test accuracy: 87.9%

# SGD ann
# Minibatch loss at step 3000: 0.473219
# Minibatch accuracy: 99.2%
# Validation accuracy: 82.5%
# Test accuracy: 90.1%

# # dropout nn dropout on logit_1 0.8
# Minibatch loss at step 8000: 0.161763
# Minibatch accuracy: 99.2%
# Validation accuracy: 84.3%
# Test accuracy: 91.6%

# # dropout nn dropout on logit_1 0.8 l2
# Minibatch loss at step 8000: 0.161763
# Minibatch accuracy: 99.2%
# Validation accuracy: 84.3%
# Test accuracy: 91.6%

# # dropout nn dropout on logit_1 0.8 logit_2 0.8
# Minibatch loss at step 8000: 0.293436
# Minibatch accuracy: 98.4%
# Validation accuracy: 82.7%
# Test accuracy: 91.5%

Initialized
Minibatch loss at step 0: 372.437378
Minibatch accuracy: 12.5%
Validation accuracy: 30.7%
Minibatch loss at step 500: 15.759850
Minibatch accuracy: 78.9%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 5.285435
Minibatch accuracy: 80.5%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 3.086450
Minibatch accuracy: 88.3%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 3.428423
Minibatch accuracy: 86.7%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 2.070216
Minibatch accuracy: 88.3%
Validation accuracy: 81.5%
Minibatch loss at step 3000: 3.804281
Minibatch accuracy: 90.6%
Validation accuracy: 79.4%
Minibatch loss at step 3500: 1.405725
Minibatch accuracy: 93.8%
Validation accuracy: 82.1%
Minibatch loss at step 4000: 5.931430
Minibatch accuracy: 86.7%
Validation accuracy: 79.8%
Minibatch loss at step 4500: 1.631713
Minibatch accuracy: 88.3%
Validation accuracy: 82.9%
Minibatch loss at step 5000: 1.828969
Minibatch accuracy: 89.8%
Validation accu